<a href="https://colab.research.google.com/github/mauricio201922/DataScience-2/blob/main/TrabalhoAula2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nomes**

- Mauricio Ramos
- Laura Sanchez

In [10]:
import numpy
from sklearn.feature_extraction.text import CountVectorizer

In [97]:
from sklearn.naive_bayes import BernoulliNB

In [64]:
import os

NEWLINE = '\n'
SKIP_FILES = set(['cmds'])

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
    for file_name in file_names:
        if file_name not in SKIP_FILES:
            file_path = os.path.join(root, file_name)
        if os.path.isfile(file_path):
            past_header, lines = False, []
            #f = open(file_path)
            f = open(file_path, mode="r", encoding="Latin-1")
            for line in f:
                if past_header:
                    lines.append(line)
                elif line == NEWLINE:
                    past_header = True
            f.close()
            yield file_path, NEWLINE.join(lines)#.decode('cp1252', 'ignore')

In [65]:
from pandas import DataFrame

def build_data_frame(path, classification):
  data_frame = DataFrame({'text': [], 'class': []})
  for file_name, text in read_files(path):
    data_frame = data_frame.append(
        DataFrame({'text': [text], 'class': [classification]}, index=[file_name]))
  return data_frame

In [66]:
HAM = 0
SPAM = 1

SOURCES = [
      ('/content/drive/MyDrive/spamassassin/spam', SPAM),
      ('/content/drive/MyDrive/spamassassin/hard_ham', HAM),
      ('/content/drive/MyDrive/spamassassin/easy_ham', HAM)
    ]

data = DataFrame({'text': [], 'class': []})
for path, classification in SOURCES:
  data = data.append(build_data_frame(path, classification))
  
data = data.reindex(numpy.random.permutation(data.index))

In [67]:
count_vectorizer = CountVectorizer()
counts = count_vectorizer.fit_transform(numpy.asarray(data['text']))

In [68]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
targets = numpy.asarray(data['class'])
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [71]:
examples = ['Free Viagra call today!', "I'm going to attend the Linux users group tomorrow."]
example_counts = count_vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions # [1, 0]

array([1., 0.])

In [72]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
  ('vectorizer',  CountVectorizer()),
  ('classifier',  MultinomialNB()) ])

pipeline.fit(numpy.asarray(data['text']), numpy.asarray(data['class']))
pipeline.predict(examples) # [1, 0]

array([1., 0.])

In [105]:
from sklearn.cross_validation import KFold

k_fold = KFold(n=len(data), n_folds=6, indices=False)
scores = []
for train_indices, test_indices in k_fold:
  train_text = numpy.asarray(data[train_indices]['text'])
  train_y    = numpy.asarray(data[train_indices]['class'])

  test_text  = numpy.asarray(data[test_indices]['text'])
  test_y     = numpy.asarray(data[test_indices]['class'])

  pipeline.fit(train_text, train_y)
  score = pipeline.score(test_text, test_y)
  scores.append(score)

score = sum(scores) / len(scores)

ModuleNotFoundError: ignored

In [99]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier',         BernoulliNB(binarize=0.0)) ])

In [106]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('tfidf_transformer',  TfidfTransformer()),
  ('classifier',         MultinomialNB()) ])

NameError: ignored

In [108]:
pipeline = Pipeline([
  ('count_vectorizer',   CountVectorizer(ngram_range=(1, 2))),
  ('classifier',         BernoulliNB(binarize=0.0)) ])

In [104]:
data

,text,class
/content/drive/MyDrive/spamassassin/hard_ham/0174.b78c73f91532150a15ced38fddbbedf4,I did more testing with your patch on top of t...,0.0
/content/drive/MyDrive/spamassassin/easy_ham/0775.380d5908cc272338fc163b1d1a748e8c,"\n\nIn a message dated 9/22/2002 11:38:01 PM, ...",0.0
/content/drive/MyDrive/spamassassin/easy_ham/1166.8a9dbee6ac67226f58a80c0993a700e2,"On Mon, Feb 04, 2002 at 02:33:29PM +0200, Pete...",0.0
/content/drive/MyDrive/spamassassin/spam/0482.d2338d94e40e6028a89c0a197ed7c470,"<HTML><FONT SIZE=2 PTSIZE=10 FAMILY=""SANSSERI...",1.0
/content/drive/MyDrive/spamassassin/easy_ham/0399.20be4fa82cc4b6342cf49fc6606c492e,Justin Mason writes:\n\n> Has anyone figured o...,0.0
...,...,...
/content/drive/MyDrive/spamassassin/easy_ham/1294.13f4a38babc2d7f26fc0f6867694cbd4,> Is it possible to use new apt to do (null) t...,0.0
/content/drive/MyDrive/spamassassin/easy_ham/0359.7f80dc1df7de3b5121e43f29f0a9e911,"On Sat, 24 Aug 2002 11:07:00 -0700, ""John Hall...",0.0
/content/drive/MyDrive/spamassassin/easy_ham/0644.465fabd80d5659bce12a75942fc62c06,"On Mon, 16 Sep 2002, Stephen D. Williams wrote...",0.0
/content/drive/MyDrive/spamassassin/spam/0160.b6b241d37fa9d5f772afca9ef30034c3,Everybody gets paid. No recruiting required.\...,1.0
